# Configure Fabric Data Agent

This notebook configures the Microsoft Fabric Data Agent using the `fabric-data-agent-sdk` library (in preview).

The notebook performs the following tasks:
1. **Install and Import Required Libraries** - Set up the necessary SDK and dependencies
2. **Variable Initialization and AI Instructions** - Configure data agent settings and define comprehensive AI instructions for analytics
3. **Initialize Data Agent Client** - Create a connection to the Data Agent service
4. **Connect to Existing Data Agent** - Establish connection to a pre-existing data agent instance
5. **Configure Lakehouse as Data Source** - Add the Lakehouse and select specific tables for AI access
6. **Configure Data Agent with AI Instructions and Few-shot Examples** - Apply AI instructions, remove existing few-shot examples, and add new query examples to improve the agent's performance
7. **Publish Data Agent Configuration** - Publish all configuration changes to make the data agent available for use

## Step 1: Install and Import Required Libraries

In [ ]:
# Install the fabric data agent SDK for programmatic management
# NOTE: THIS WILL BE CONFIGURED VIA ENVIRONMENT AS SCHEDULED JOBS DO NOT ALLOW pip install COMMANDS
# %pip install fabric-data-agent-sdk==0.1.16a0
# %pip show fabric-data-agent-sdk

# Import required libraries
from uuid import UUID
from fabric.dataagent.client import FabricDataAgentManagement

print("✅ Installation and import complete")

## Step 2: Variable Initialization and AI Instructions

Configure the variables needed for the data agent setup and define AI instructions:

In [ ]:
# Configuration variables
data_agent_id = "__AGENT_ID__"
lakehouse_id = "__LAKEHOUSE_ID__"
lakehouse_workspace_id = "__LAKEHOUSE_WORKSPACE_ID__"

# Configure specific tables to be available to the AI
# Note: For Lakehouse tables, use schema (e.g., "dbo") and table name
selected_tables = __SELECTED_TABLES__

print(f"📋 Configuration:")
print(f"   Data Agent ID: {data_agent_id}")
print(f"   Lakehouse ID: {lakehouse_id}")

# AI instructions
agent_instructions = """
__AGENT_INSTRUCTIONS__
"""

data_source_instructions="""
__DATA_SOURCE_INSTRUCTIONS__
```
"""

data_source_description="""
__DATA_SOURCE_DESCRIPTION__
"""

# Initialize few-shot examples for queries based on your data
fewshots_examples = __QUERY_EXAMPLES__

print(f"📋 AI Instructions and Configuration Defined:")
print(f"   Agent Instructions: {len(agent_instructions)} characters")
print(f"   Data Source Description: {len(data_source_description)} characters")
print(f"   Data Source Instructions: {len(data_source_instructions)} characters")
print(f"   Fewshots Examples: {len(fewshots_examples)} examples prepared")
print(f"   Selected Tables: {len(selected_tables)} tables configured")
print(f"   ✅ Configuration ready")

## Step 3: Initialize Data Agent Client

Create a connection to the Data Agent service:

In [ ]:
# Initialize the Data Agent management client for existing data agent
mgmt_client = FabricDataAgentManagement(UUID(data_agent_id))
print(f"✅ Successfully initialized Data Agent management client for: {data_agent_id}")
print(f"✅ Client ready for data agent operations")

## Step 4: Connect to Existing Data Agent

Connect to an existing data agent using the configured ID:

In [ ]:
# Connect to existing data agent and verify configuration
print(f"🤖 Connecting to existing data agent: {data_agent_id}")

config = mgmt_client.get_configuration()
print(f"✅ Successfully connected to data agent")

print(f"\n📊 Data Agent Details:")
print(f"   Name: Data Agent")
print(f"   ID: {data_agent_id}")
print(f"   Status: Ready for configuration")

## Step 5: Configure Lakehouse as Data Source

Add the Lakehouse as a data source and select specific tables for the data agent:

In [ ]:
# Add Lakehouse as data source to the data agent
print(f"🔗 Adding Lakehouse as data source...")
print(f"   Data Agent ID: {data_agent_id}")
print(f"   Lakehouse ID: {lakehouse_id}")

# Add the Lakehouse as a data source
datasource = mgmt_client.add_datasource(
    workspace_id_or_name=UUID(lakehouse_workspace_id),
    artifact_name_or_id=UUID(lakehouse_id),
    type="lakehouse"
)

print(f"✅ Successfully added Lakehouse data source")
print(f"   Datasource ID: {datasource._id}")

# Configure table selection from config variables
print(f"\n📋 Configuring table selection...")
print(f"   Selected tables: {', '.join([f'{schema}.{table}' for schema, table in selected_tables])}")

# Enable the specified tables for the data agent
for schema, table_name in selected_tables:
    datasource.select(schema, table_name)
    print(f"   ✓ Enabled table: {schema}.{table_name}")

print(f"✅ Table configuration completed")
print(f"   Tables available to AI: {', '.join([f'{schema}.{table}' for schema, table in selected_tables])}")

## Step 6: Configure Data Agent with AI Instructions and Few-shot Examples

Apply the AI instructions and add few-shot examples to configure the data agent's behavior:

In [ ]:
# Update data agent with general AI instructions
print(f"🤖 Updating data agent with AI instructions...")
print(f"   Instructions length: {len(agent_instructions)} characters")

mgmt_client.update_configuration(instructions=agent_instructions)
print(f"✅ Successfully updated data agent configuration")

# Configure data source with specific instructions and description
print(f"\n🔗 Configuring data source instructions...")
print(f"   Instructions length: {len(data_source_instructions)} characters")

datasource.update_configuration(
    instructions=data_source_instructions,
    user_description=data_source_description
)
print(f"✅ Successfully updated data source configuration")

# Get existing few-shot examples and remove them
print(f"\n🔍 Checking for existing few-shot examples...")
existing_fewshots = datasource.get_fewshots()
print(f"   Found {len(existing_fewshots)} existing examples")

if len(existing_fewshots) > 0:
    print(f"🗑️ Removing existing few-shot examples...")
    for i, row in existing_fewshots.iterrows():
        fewshot_id = row['Id']
        question = row['Question'][:50] + ('...' if len(row['Question']) > 50 else '')
        print(f"   Removing: {question}")
        datasource.remove_fewshot(fewshot_id)
    print(f"✅ Successfully removed all {len(existing_fewshots)} existing examples")
else:
    print(f"   No existing examples to remove")

# Add few-shot examples to improve query generation
print(f"\n📚 Adding few-shot examples...")
print(f"   Adding {len(fewshots_examples)} example question-query pairs")

for i, (question, query) in enumerate(fewshots_examples.items(), 1):
    print(f"   {i}. Adding: {question[:60]}{'...' if len(question) > 60 else ''}")
    single_example = {question: query}
    datasource.add_fewshots(single_example)

print(f"✅ Successfully added all {len(fewshots_examples)} few-shot examples")

# Verify final configuration
fewshots_df = datasource.get_fewshots()
config = mgmt_client.get_configuration()
ds_config = datasource.get_configuration()

print(f"\n📊 Final Configuration Summary:")
print(f"   Agent instructions: {'✓' if config.instructions else '✗'}")
print(f"   Data source instructions: {'✓' if ds_config.get('additional_instructions') else '✗'}")
print(f"   Data source description: {'✓' if ds_config.get('user_description') else '✗'}")
print(f"   Few-shot examples: {len(fewshots_df)}")
print(f"   Datasource ID: {datasource._id}")

print(f"\n✅ Data agent configuration completed successfully!")

## Step 7: Publish Data Agent Configuration

Publish the data agent configuration to make it available for use:

In [ ]:
# Publish the data agent configuration
print(f"📤 Publishing data agent configuration...")
print(f"   Making data agent available for use...")

mgmt_client.publish()
print(f"✅ Successfully published data agent configuration")
print(f"   Data agent is now ready to answer questions!")
print(f"   You can now interact with the agent in Fabric using natural language queries")

print(f"\n🎉 Data Agent Configuration Complete!")
print(f"   Agent ID: {data_agent_id}")
print(f"   Status: Published and Ready")
print(f"   Available Tables: {', '.join([f'{schema}.{table}' for schema, table in selected_tables])}")
print(f"   Few-shot Examples: {len(fewshots_examples)}")
print(f"   Next: Test the agent with analytics queries!")